### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

# Goal-oriented bot in DeepPavlov

The tutor is focused on building a goal-oriented dialogue system:

0. [Data preparation](#0.-Data-Preparation)
1. [Build database of items](#1.-Build-database-of-items)
2. [Build Slot Filler](#2.-Build-Slot-Filler)
3. [Train bot](#3.-Train-bot)

An example of the final model served as a telegram bot is:

![gobot_example.png](img/gobot_example.png)

## 0. Data Preparation

The tutor's dialogue system will be on the domain of restaurant booking. [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/) dataset provides dialogues of a human talking to a booking system labelled with slots and dialogue actions. The labels are will be used for training a dialogue policy network.

See below a small chunk of the data. 

In [1]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-09-04 14:40:33.370 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [PosixPath('my_data/simple-dstc2-val.json'), PosixPath('my_data/simple-dstc2-tst.json')]]
2019-09-04 14:40:33.371 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 270: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2019-09-04 14:40:33.399 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:00<00:00, 67.5MB/s]
2019-09-04 14:40:33.410 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2019-09-04 14:40:33.442 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-09-04 14:40:33.534 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['ds

In [2]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


The training/validation/test data is stored in json files (`simple-dstc2-trn.json`, `simple-dstc2-val.json` and `simple-dstc2-tst.json`):

In [3]:
!head -n 101 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },
    {
      "speaker": 1,
      "text": "any",
      "slots": [
        [
          "this",
          "dontcare"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What part of town do you have in mind?",
      "slots": [],
      "act": "request_area"
    },
    {
      "speaker": 1,
      "text": "south",
      "slots": [
        [
          "area",
          "south"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "api_call area=\"south\" food

In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed DSTC-2 dialogs:

In [5]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {   'prev_resp_act': 'welcomemsg',
        'slots': [['pricerange', 'moderate'], ['area', 'north']],
        'text': 'im looking for a moderately priced restaurant in the north '
                'part of town'},
    {   'db_result': {   'addr': '7 milton road chesterton',
                         'area': 'north',
                         'food': 'indian',
                         'name': 'the nirala',
                         'phone': '01223 360966',
                         'postcode': 'c.b 4, 1 u.y',
                         'pricerange': 'moderate'},
        'prev_resp_act': 'api_call',
        'slots': [['pricerange', 'moderate'], ['area', 'north']],
        'text': 'im looking for a moderately priced restaurant in the north '
                'part of town'},
    {   'prev_resp_act': 'inform_area+inform_pricerange+offer_name',
        'slots': [['slot', 'phone']],
        'text': 'what is the phone numb

In [6]:
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [7]:
import json

NUM_TRAIN = 50

with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin:
    data = json.load(fin)
with open('my_data/simple-dstc2-trn.json', 'wt') as fout:
    json.dump(data[:NUM_TRAIN], fout, indent=2)
print(f"Train set is reduced to {NUM_TRAIN} dialogues (out of {len(data)}).")

Train set is reduced to 50 dialogues (out of 967).


## 1. Build database of items

&nbsp;
![gobot_database.png](img/gobot_database.png)
&nbsp;

For a valid goal-oriented bot there should be a `database` of relevant items. In the case of restaurant booking it will contain all available restaurants and their info.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           
The dialogues in the training dataset should contain a `"db_result"` dictionary key. It is required for turns where system performs a special type of external action: an api call to the database of items. `"db_result"` should contain the result of the api call:

In [8]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

    {
      "speaker": 2,
      "text": "api_call area=\"south\" food=\"dontcare\" pricerange=\"cheap\"",
      "db_result": {
        "food": "chinese",
        "pricerange": "cheap",
        "area": "south",
        "addr": "cambridge leisure park clifton way cherry hinton",
        "phone": "01223 244277",
        "postcode": "c.b 1, 7 d.y",
        "name": "the lucky star"
      },
      "slots": [
        [
          "area",
          "south"
        ],
        [
          "pricerange",
          "cheap"
        ],
        [
          "food",
          "dontcare"
        ]
      ],
      "act": "api_call"
    },


In [9]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

2019-09-04 14:40:49.312 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-04 14:40:49.313 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 70: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


Set `primary_keys` to a list of slot names that have unique values for different items (common SQL term). For the case of DSTC-2, the primary slot is restaurant name.

Let's find all `"db_result"` api call results and add it to our database of restaurants:

In [10]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-09-04 14:40:50.332 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 145: Created table with keys {'food': 'text', 'postcode': 'text', 'pricerange': 'text', 'area': 'text', 'phone': 'text', 'name': 'text', 'addr': 'text'}.


Adding 3016 items.


##### Interacting with database

We can now play with the database and make requests to it:

In [11]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'food': 'chinese',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap',
   'area': 'south',
   'phone': '01223 244277',
   'name': 'the lucky star',
   'addr': 'cambridge leisure park clifton way cherry hinton'},
  {'food': 'portuguese',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap',
   'area': 'south',
   'phone': '01223 327908',
   'name': 'nandos',
   'addr': 'cambridge leisure park clifton way'}]]

In [12]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

&nbsp;
![gobot_slotfiller.png](img/gobot_slotfiller.png)
&nbsp;

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

In [13]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-09-04 14:40:53.225 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 11.1MB/s]
2019-09-04 14:40:53.227 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [14]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [15]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


##### Metric scores on valid&test

Let's check performance of our slot filler on DSTC-2 dataset:

In [16]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

We take [original DSTC2 slot-filling config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json) and change variables determining data paths:

In [17]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [18]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

2019-09-04 14:40:55.992 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-04 14:40:55.999 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-04 14:40:56.105 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-04 14:40:56.150 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 479 samples in train split.
2019-09-04 14:40:56.151 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-04 14:40:56.151 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

{"valid": {"eval_examples_count": 1253, "metrics": {"slots_accuracy": 0.933}, "time_spent": "0:00:34"}}
{"test": {"eval_examples_count": 1190, "metrics": {"slots_accuracy": 0.9487}, "time_spent": "0:00:31"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

##### Interacting with slot filler

In [19]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

In [20]:
slotfill(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

##### Dumping slot filler's config

Saving slotfill config file to disk (we will require it's path later):

In [21]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

In [22]:
!ls my_bot

db.sqlite  slotfill  slotfill_config.json


## 3. Train bot

Let's assemble all modules together and train the final module: dialogue policy network.

&nbsp;
![gobot_policy.png](img/gobot_policy.png)
&nbsp;

Policy network decides which action the system should take on each turn of a dialogue: should it say goodbye, request user's location or make api call to a database.

The policy network is a recurrent neural network (recurrent over utterances represented as bags of words) and a dense layer with softmax function on top. The network classifies user utterance into one of predefined system actions.

&nbsp;
![gobot_templates.png](img/gobot_templates.png)
&nbsp;

All actions available for the system should be listed in a `simple-dstc2-templates.txt` file. Each action should be associated with a string of the corresponding system response.

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response. Response text might contain slot type names, where every `#slot_type` will be filled with the slot value from a dialogue state.

In [23]:
!head -n 10 my_data/simple-dstc2-templates.txt

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.
canthelp_area_food_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #food food.
canthelp_area_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #area american food.
canthelp_food	I am sorry but there is no #food restaurant that matches your request.
canthelp_food_pricerange	Sorry there is no #food restaurant in the #pricerange price range.
confirm-domain	You are looking for a restaurant is that right?


So, actions are actually classes we classify over. And `simple-dstc2-templates.txt` contains the set of classes.

To train the dialogue policy network for classification task you need action label for each system utterance in training dialogues. The DSTC-2 contains `"act"` dictionary key that contains action associated with current response.

The cell below provides an example of training data for the policy network.

In [24]:
!head -n 24 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },


Let's **construct the final pipeline** of a dialogue system.

We take [default DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_dstc2.json) ([more configs](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) are available) and change sections responsible for 
- templates,
- database,
- slot filler,
- embeddings,
- data and model load/save paths.

In [25]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

**Configure** bot to use **templates**:

In [26]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my_data/simple-dstc2-templates.txt'

**Configure** bot to use our built **database**:

In [27]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [28]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

You can use a simple **bag-of-words as embedder** (by default):

In [29]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [30]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_pipeline.png](img/gobot_pipeline.png)

##### Training policy network

In [31]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['max_batches'] = 250 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 40 # evaluate on full 'valid' split each 30 batches
gobot_config['train']['log_every_n_batches'] = 40 # evaluate on 20 batches of 'train' split every 30 batches
gobot_config['train']['log_on_k_batches'] = 20

train_model(gobot_config);

2019-09-04 14:43:07.476 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-04 14:43:07.482 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-04 14:43:07.663 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-04 14:43:07.718 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 479 samples in train split.
2019-09-04 14:43:07.718 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-04 14:43:07.719 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.
2019-09-04 14:43:09.274 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-04 14:43:09.275 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-04 14:43:09.276 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 205


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


2019-09-04 14:43:10.56 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 251: [initializing `GoalOrientedBot` from scratch]
2019-09-04 14:44:50.985 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.0072
2019-09-04 14:44:50.986 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
2019-09-04 14:44:50.987 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-09-04 14:44:51.75 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 763: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]


...

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model
{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4113}, "time_spent": "0:01:41"}}


2019-09-04 15:06:47.967 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-04 15:06:47.969 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-04 15:06:47.970 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-04 15:06:48.86 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-04 15:06:48.87 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-04 15:06:48.87 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 205


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4065}, "time_spent": "0:01:38"}}


2019-09-04 15:06:48.683 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
2019-09-04 15:06:48.684 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 769: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-09-04 15:06:48.686 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model


Training on 50 dialogues takes from 5 to 20 minutes depending on gpu/cpu. Training on full data takes 10-30 mins.

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [32]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-09-04 15:06:48.730 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-04 15:06:48.736 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-04 15:06:48.782 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-04 15:06:48.996 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 479 samples in train split.
2019-09-04 15:06:48.997 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-04 15:06:48.997 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model
{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4113}, "time_spent": "0:01:41"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4065}, "time_spent": "0:01:38"}}


With settings of `max_batches=200`, valid accuracy `~ 0.4` and test accuracy is `0.4`.

##### Chatting with bot

In [33]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-09-04 15:10:08.45 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-04 15:10:08.46 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-04 15:10:08.47 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-04 15:10:08.158 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-templates.txt]
2019-09-04 15:10:08.159 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 46 templates loaded.
2019-09-04 15:10:08.159 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 205
2

INFO:tensorflow:Restoring parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model


In [61]:
bot(['hi, i want to eat, can you suggest a place to go?'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [62]:
bot(['i want cheap food'])

['What kind of food would you like?']

In [63]:
bot(['chinese food'])

2019-09-04 15:14:33.854 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 491: Made api_call with {'pricerange': 'cheap', 'food': 'chinese'}, got 4 results.


['The lucky star serves chinese food.']

In [64]:
bot(['thanks, give me their address'])

['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']

In [66]:
bot(['i want their phone number too'])

['The phone number of the lucky star is 01223 244277.']

In [39]:
bot(['bye'])

['You are welcome!']

In [68]:
bot.reset()

In [69]:
bot(['hi, is there any cheap restaurant?'])

['What kind of food would you like?']